In [ ]:
from ultralytics import YOLO
import torch.nn as nn
import torch

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# YOLO의 Detect 모듈 수정: 마지막 레이어를 6개의 파라미터 (cx, cy, w, h, angle)로 확장
class RotatedYOLO(nn.Module):
    def __init__(self, base_model):
        super(RotatedYOLO, self).__init__()
        self.base_model = base_model

        # YOLO의 Detect 레이어 가져오기
        self.detect_layer = self.base_model.model.model[-1]

        # YOLO의 기존 클래스 개수
        num_classes = self.detect_layer.nc

        # 새로운 Conv 레이어 추가: (cx, cy, w, h, angle) + obj_conf + class_probs
        in_channels = self.detect_layer.cv3[-1][2].out_channels  # 마지막 Conv 레이어의 출력 채널
        out_channels = 5 + 1 + num_classes  # (cx, cy, w, h, angle, obj_conf + class_probs)

        # 새로운 Conv2d 레이어 생성
        self.new_conv = nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1))

        # Detect 레이어의 cv3 수정
        self.detect_layer.cv3[-1].add_module('new_conv', self.new_conv)

    def forward(self, x):
        # YOLO의 forward 과정을 수행
        return self.base_model(x)

    def train(self, *args, **kwargs):
        # YOLO의 기본 train 메소드 호출
        self.base_model.train(*args, **kwargs)

        # 필요한 커스텀 손실 함수를 train 루틴에 포함하기 위해 YOLO의 train 메소드 수정
        self.base_model.loss = self.custom_loss  # 손실 함수를 커스텀으로 변경

    # 커스텀 손실 함수 정의
    def custom_loss(self, preds, targets):
        # 바운딩 박스 손실 계산 (cx, cy, w, h)
        bbox_loss = nn.MSELoss()(preds[:, :4], targets[:, :4])  # (cx, cy, w, h)

        # 각도 손실 계산 (angle)
        angle_loss = nn.MSELoss()(preds[:, 4], targets[:, 4])  # angle

        # 총 손실 계산 (기존 바운딩 박스 손실 + 각도 손실)
        total_loss = bbox_loss + 0.1 * angle_loss  # 0.1은 각도 손실 가중치
        return total_loss


# 기존 모델에 수정된 구조 적용
rotated_model = RotatedYOLO(model)

# 학습 시작 (rotated_model을 사용하여 훈련)
results = rotated_model.train(
    data='data.yaml',  # 데이터 설정 파일
    epochs=50,         # 학습 epoch 수
    imgsz=256,         # 입력 이미지 크기
    batch=16,          # 배치 크기
    name='rotated_ship_detection',  # 실험 이름
    device=0           # GPU 사용 설정
)


In [14]:
import torch
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11m.pt")

# 모델의 전체 구조를 요약해서 출력
print(model.model)

# 임의의 입력 텐서 생성 (배치 크기 1, 채널 수 3, 높이 256, 너비 256)
input_tensor = torch.randn(1, 3, 256, 256)

# 모델을 통과시켜서 출력 얻기
with torch.no_grad():  # 기울기 계산을 하지 않도록 설정
    outputs = model(input_tensor)

# 출력의 형태와 타입을 출력
print("Outputs:", outputs)  # 출력 내용을 확인

# 각 출력의 타입 및 형태 확인
if isinstance(outputs, list):  # outputs가 리스트인지 확인
    for i, output in enumerate(outputs):
        print(f"Output {i}: Type = {type(output)}, Value = {output}")
else:
    print("Output is not a list.")


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   

In [8]:

# 모델의 레이어 이름과 모듈을 모두 출력
for name, module in model.model.named_modules():
    print(f"Layer Name: {name}, Module: {module}")


Layer Name: , Module: DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): 

In [10]:
# YOLO의 Detect 모듈 출력
detect_layer = model.model.model[-1]  # Detect 레이어에 접근
print(detect_layer)


Detect(
  (cv2): ModuleList(
    (0): Sequential(
      (0): Conv(
        (conv): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1-2): 2 x Sequential(
      (0): Conv(
        (conv): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)